In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd "/content/drive/MyDrive/deep-learning/kmean/Images"

/content/drive/MyDrive/deep-learning/kmean/Images


In [4]:
pip install opencv-python

Khai báo thư viện

In [11]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

Đọc ảnh ban đầu

In [13]:
def read_image(image_path):
    image = cv2.imread(image_path)
    return image

In [14]:
img = read_image("myphoto.jpg")
cv2_imshow(img)

Output hidden; open in https://colab.research.google.com to view.

Chuyển ảnh thành ma trận 2D (Mỗi pixel là ma trận 3 chiều)

In [16]:
def reshape_image(image):
    pixels = image.reshape((-1, 3))
    pixels = np.float32(pixels) # Chuyển kiểu fload 32 để tính toán chuẩn hơn cho màu ảnh của từng pixel
    return pixels

In [18]:
pexels = reshape_image(img)
pexels

array([[ 77., 138., 110.],
       [ 93., 157., 128.],
       [ 78., 145., 114.],
       ...,
       [102., 123., 185.],
       [101., 122., 184.],
       [100., 121., 183.]], dtype=float32)

In [20]:
pexels.shape

(1228800, 3)

Khởi tạo ngẫu nhiên k cụm

In [24]:
def initialize_centroids(pixels, k):
    n_pixels = len(pixels)
    random_indices = np.random.choice(n_pixels, k, replace=False)
    centroids = pixels[random_indices]
    return centroids

In [25]:
centroids = initialize_centroids(pexels, 20)
centroids

array([[ 27.,  27.,  27.],
       [ 74.,  73.,  69.],
       [ 56.,  85.,  92.],
       [  0.,   2.,  10.],
       [122., 133., 153.],
       [ 82., 103., 105.],
       [ 85.,  81.,  86.],
       [ 36.,  52.,  69.],
       [ 43.,  49.,  54.],
       [ 91., 107., 119.],
       [ 20.,  30.,  40.],
       [ 69.,  80.,  84.],
       [132., 162., 157.],
       [156., 207., 199.],
       [115., 153., 131.],
       [ 95., 139., 138.],
       [113., 143., 132.],
       [155., 146., 179.],
       [112., 128., 135.],
       [ 13.,  42.,  16.]], dtype=float32)

Gán pixel vào cụm gần nhất

In [ ]:
def assign_pixels_to_clusters(pixels, centroids):
    distances = np.sqrt(((pixels - centroids[:, np.newaxis])**2).sum(axis=2))
    return np.argmin(distances, axis=0)

Cập nhật vị trí các cụm

In [22]:
def update_centroids(pixels, labels, k):
    new_centroids = np.array([pixels[labels == i].mean(axis=0) for i in range(k)])
    return new_centroids

In [9]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

def kmeans_image_segmentation(image_path, k, max_iters=100):
    """
    Thực hiện phân đoạn ảnh sử dụng thuật toán K-means

    Parameters:
        image_path: Đường dẫn đến ảnh
        k: Số lượng cụm màu
        max_iters: Số lần lặp tối đa
    Returns:
        segmented_image: Ảnh đã được phân đoạn
        labels: Nhãn của từng pixel
        centers: Các tâm cụm màu
    """
    # Đọc ảnh
    image = cv2.imread(image_path)

    # Chuyển đổi ảnh sang không gian màu LAB
    # LAB color space thường cho kết quả phân đoạn tốt hơn RGB
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    # Reshape ảnh thành mảng 2D (mỗi pixel là một vector 3 chiều)
    pixels = lab_image.reshape((-1, 3))

    # Chuyển đổi sang kiểu float32
    pixels = np.float32(pixels)

    # Định nghĩa điều kiện dừng
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, max_iters, 1.0)

    # Thực hiện K-means
    _, labels, centers = cv2.kmeans(
        data=pixels,
        K=k,
        bestLabels=None,
        criteria=criteria,
        attempts=10,
        flags=cv2.KMEANS_RANDOM_CENTERS
    )

    # Chuyển centers về kiểu uint8
    centers = np.uint8(centers)

    # Gán lại màu cho từng pixel theo tâm cụm
    segmented_pixels = centers[labels.flatten()]

    # Reshape lại kích thước ảnh gốc
    segmented_image = segmented_pixels.reshape(lab_image.shape)

    # Chuyển đổi lại sang BGR
    segmented_image = cv2.cvtColor(segmented_image, cv2.COLOR_LAB2BGR)

    return segmented_image, labels, centers

def apply_color_quantization(image_path, k, output_path=None):
    """
    Áp dụng color quantization và lưu kết quả

    Parameters:
        image_path: Đường dẫn ảnh đầu vào
        k: Số lượng màu mong muốn
        output_path: Đường dẫn lưu ảnh kết quả (tùy chọn)
    """
    # Thực hiện phân đoạn
    segmented_image, labels, centers = kmeans_image_segmentation(image_path, k)

    # Hiển thị ảnh gốc
    original = cv2.imread(image_path)
    cv2_imshow( original)

    # Hiển thị ảnh đã phân đoạn
    cv2_imshow( segmented_image)

    # Lưu ảnh nếu có đường dẫn output
    if output_path:
        cv2.imwrite(output_path, segmented_image)
        print(f"Đã lưu ảnh kết quả tại: {output_path}")

    # Hiển thị bảng màu
    # Tạo bảng màu từ các tâm cụm
    color_palette = np.zeros((50, k * 50, 3), dtype=np.uint8)
    for i in range(k):
        color_palette[:, i*50:(i+1)*50] = centers[i]
    color_palette = cv2.cvtColor(color_palette, cv2.COLOR_LAB2BGR)
    cv2_imshow(color_palette)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

def analyze_color_distribution(labels, k):
    """
    Phân tích phân phối màu trong ảnh

    Parameters:
        labels: Nhãn của từng pixel từ K-means
        k: Số lượng cụm màu
    """
    # Tính tỉ lệ phần trăm của từng màu
    total_pixels = len(labels)
    color_distribution = []

    for i in range(k):
        color_count = np.count_nonzero(labels == i)
        percentage = (color_count / total_pixels) * 100
        color_distribution.append(percentage)
        print(f"Màu {i+1}: {percentage:.2f}%")

    return color_distribution

In [10]:
image_path = "myphoto.jpg"  # Đường dẫn ảnh đầu vào
output_path = "output.jpg"  # Đường dẫn lưu ảnh kết quả
k = 5  # Số lượng màu mong muốn

# Thực hiện phân đoạn và hiển thị kết quả
apply_color_quantization(image_path, k, output_path)

# Nếu muốn phân tích phân phối màu
segmented_image, labels, centers = kmeans_image_segmentation(image_path, k)
color_distribution = analyze_color_distribution(labels, k)

Output hidden; open in https://colab.research.google.com to view.